In [1]:
import json

import evaluate
import torch as tt
import pandas as pd

from typing import Any, Dict, Union

from nltk.tokenize import sent_tokenize
from tqdm import tqdm_notebook
from datasets import Dataset, load_dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, PreTrainedModel, PreTrainedTokenizer
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments

In [13]:
s1 = """Наш корабль стоял на якоре у берега Африки. День был прекрасный, с моря дул свежий ветер; но к вечеру погода изменилась: стало душно и точно из топленной печки несло на нас горячим воздухом с пустыни Сахары. Перед закатом солнца капитан вышел на палубу, крикнул: «Купаться!» — и в одну минуту матросы попрыгали в воду, спустили в воду парус, привязали его и в парусе устроили купальню.
На корабле с нами было два мальчика. Мальчики первые попрыгали в воду, но им тесно было в парусе, они вздумали плавать наперегонки в открытом море.
Оба, как ящерицы, вытягивались в воде и что было силы поплыли к тому месту, где был бочонок над якорем.
Один мальчик сначала перегнал товарища, но потом стал отставать. Отец мальчика, старый артиллерист, стоял на палубе и любовался на своего сынишку. Когда сын стал отставать, отец крикнул ему: «Не выдавай! Понатужься!»
Вдруг с палубы кто-то крикнул: «Акула!» — и все мы увидали в воде спину морского чудовища.
Акула плыла прямо на мальчиков.
— Назад! Назад! Вернитесь! Акула! — закричал артиллерист. Но ребята не слыхали его, плыли дальше, смеялись и кричали еще веселее и громче прежнего.
Артиллерист, бледный как полотно, не шевелясь, смотрел на детей.
Матросы спустили лодку, бросились в нее и, сгибая весла, понеслись что было силы к мальчикам; но они были еще далеко от них, когда акула уже была не дальше 20-ти шагов.
Мальчики сначала не слыхали того, что им кричали, и не видали акулы; но потом один из них оглянулся, и мы все услыхали пронзительный визг, и мальчики поплыли в разные стороны.
Визг этот как будто разбудил артиллериста. Он сорвался с места и побежал к пушкам. Он повернул хобот, прилег к пушке, прицелился и взял фитиль.
Мы все, сколько нас ни было на корабле, замерли от страха и ждали, что будет.
Раздался выстрел, и мы увидали, что артиллерист упал подле пушки и закрыл лицо руками. Что сделалось с акулой и с мальчиками, мы не видали, потому что на минуту дым застлал нам глаза.
Но когда дым разошелся над водою, со всех сторон послышался сначала тихий ропот, потом ропот этот стал сильнее, и, наконец, со всех сторон раздался громкий, радостный крик.
Старый артиллерист открыл лицо, поднялся и посмотрел на море.
По волнам колыхалось желтое брюхо мертвой акулы. В несколько минут лодка подплыла к мальчикам и привезла их на корабль. ВОПРОС: Какое высказывание СООТВЕТСТВУЕТ тексту? ПРАВИЛЬНЫЙ ОТВЕТ: "Меткость старого артиллериста спасла мальичков от морского чудовища" НЕПРАВИЛЬНЫЕ ВАРИАНТЫ ОТВЕТА:"""

In [3]:
# models:
tokenizer = T5Tokenizer.from_pretrained("ai-forever/ruT5-base")
model = T5ForConditionalGeneration.from_pretrained("RuT5-RACE-tf-1/checkpoint-65760").to(tt.device("cuda:0"))

# metrics:
bleu4 = evaluate.load("bleu")
sbleu = evaluate.load("sacrebleu")
rouge = evaluate.load("rouge")
meteor = evaluate.load("meteor")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
[nltk_data] Downloading package wordnet to /home/user/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/user/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [7]:
def model_predict(text: str, model: PreTrainedModel) -> str:
    input_ = tokenizer([text], return_tensors="pt")
    output = model.generate(
        input_["input_ids"].to(tt.device("cuda:0")),
        max_length=81
    )
    return tokenizer.batch_decode(output)[0].replace("<pad>", " ").replace("</s>", " ").strip()

In [14]:
predict_model = model_predict(s1, model)

In [9]:
predict_model

'"Старый артиллерист отвел мальчика в сторону."; "Мальчики, которые были сбиты в лодке, не слышали крика старого артиллериста."; "История произошла в день, когда мы увидели акулу."'

In [15]:
s2 = """Наш корабль стоял на якоре у берега Африки. День был прекрасный, с моря дул свежий ветер; но к вечеру погода изменилась: стало душно и точно из топленной печки несло на нас горячим воздухом с пустыни Сахары. Перед закатом солнца капитан вышел на палубу, крикнул: «Купаться!» — и в одну минуту матросы попрыгали в воду, спустили в воду парус, привязали его и в парусе устроили купальню.
На корабле с нами было два мальчика. Мальчики первые попрыгали в воду, но им тесно было в парусе, они вздумали плавать наперегонки в открытом море.
Оба, как ящерицы, вытягивались в воде и что было силы поплыли к тому месту, где был бочонок над якорем.
Один мальчик сначала перегнал товарища, но потом стал отставать. Отец мальчика, старый артиллерист, стоял на палубе и любовался на своего сынишку. Когда сын стал отставать, отец крикнул ему: «Не выдавай! Понатужься!»
Вдруг с палубы кто-то крикнул: «Акула!» — и все мы увидали в воде спину морского чудовища.
Акула плыла прямо на мальчиков.
— Назад! Назад! Вернитесь! Акула! — закричал артиллерист. Но ребята не слыхали его, плыли дальше, смеялись и кричали еще веселее и громче прежнего.
Артиллерист, бледный как полотно, не шевелясь, смотрел на детей.
Матросы спустили лодку, бросились в нее и, сгибая весла, понеслись что было силы к мальчикам; но они были еще далеко от них, когда акула уже была не дальше 20-ти шагов.
Мальчики сначала не слыхали того, что им кричали, и не видали акулы; но потом один из них оглянулся, и мы все услыхали пронзительный визг, и мальчики поплыли в разные стороны.
Визг этот как будто разбудил артиллериста. Он сорвался с места и побежал к пушкам. Он повернул хобот, прилег к пушке, прицелился и взял фитиль.
Мы все, сколько нас ни было на корабле, замерли от страха и ждали, что будет.
Раздался выстрел, и мы увидали, что артиллерист упал подле пушки и закрыл лицо руками. Что сделалось с акулой и с мальчиками, мы не видали, потому что на минуту дым застлал нам глаза.
Но когда дым разошелся над водою, со всех сторон послышался сначала тихий ропот, потом ропот этот стал сильнее, и, наконец, со всех сторон раздался громкий, радостный крик.
Старый артиллерист открыл лицо, поднялся и посмотрел на море.
По волнам колыхалось желтое брюхо мертвой акулы. В несколько минут лодка подплыла к мальчикам и привезла их на корабль. ВОПРОС: Какое высказывание НЕ СООТВЕТСТВУЕТ тексту? ПРАВИЛЬНЫЙ ОТВЕТ: "Храбрость помогла мальчикам спастись от акулы" НЕПРАВИЛЬНЫЕ ВАРИАНТЫ ОТВЕТА:"""

In [16]:
predict_model = model_predict(s2, model)

In [17]:
predict_model

'"На корабле было много акул."; "На корабле было много акул."; "Капитан знал, что акула напали не на мальчиков, а на корабль."'